In [1]:
%matplotlib inline

In [2]:
import os 
import sys
import json
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import neurokit as nk
import scipy as sc
import math 
import scipy.signal as ss
import warnings
import itertools

from scipy import signal
warnings.filterwarnings('ignore')

In [3]:
path = "/mnt/shared_drive/data/brain_therapy/Brain_Therapy_MVP_Content_Pilot/FA_corrected_signals"
files = os.listdir(path)

In [ ]:
sensor_data = pd.DataFrame([])
for file in files:
    data = pd.read_csv(os.path.join(path,file))
    data['Participant'] = file.split("-")[3].split(".")[0]
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    data.set_index(['Participant','Datetime'], inplace = True)
    sensor_data = pd.concat([sensor_data,data])

In [ ]:
participants = sensor_data.index.get_level_values('Participant').unique().values
participants = sorted(participants)
sensor_data.sort_index(sort_remaining = True, inplace = True)
sensor_data.head()

In [4]:
log_events = joblib.load("/mnt/shared_drive/data/brain_therapy/log_events.pkl")

In [5]:
log_events = log_events.reset_index().rename(columns = {'participant':"Participant", "time":"Datetime"}).set_index(["Participant","Datetime"])
log_events.head()

type           context
Participant Datetime                                  
0001        2019-07-10 15:25:32   2D  StartFrontBuffer
            2019-07-10 15:27:32   2D   StopFrontBuffer
            2019-07-10 15:27:32   2D           StartLL
            2019-07-10 15:29:42   2D            StopLL
            2019-07-10 15:29:42   2D           StartHH

### Using the curated signals to compute features

In [6]:
curated_signals = joblib.load("/mnt/shared_drive/data/brain_therapy/Brain_Therapy_MVP_Content_Pilot/FA_corrected_signals_amplitude.pkl")

In [7]:
p = curated_signals.keys()
curated_df = pd.DataFrame([])
for part in p:
    data = curated_signals[part]
    data['Participant'] = part.split("-")[3]
    curated_df = pd.concat([curated_df,data])
curated_df.set_index(["Participant","Datetime"], inplace = True)
curated_df.head()

RSP       PPG     1-SKTA       ECG  \
Participant Datetime                                                           
0001        2019-07-10 15:22:31.000 -1.263123  0.003052  12.941162 -0.234833   
            2019-07-10 15:22:31.004 -1.261902  0.003662  12.945435 -0.226593   
            2019-07-10 15:22:31.008 -1.264648  0.002136  12.946045 -0.215607   
            2019-07-10 15:22:31.012 -1.261902  0.003662  12.945435 -0.206604   
            2019-07-10 15:22:31.016 -1.262207  0.003357  12.946045 -0.198364   

                                          EDA  
Participant Datetime                           
0001        2019-07-10 15:22:31.000  0.041124  
            2019-07-10 15:22:31.004  0.042649  
            2019-07-10 15:22:31.008  0.044175  
            2019-07-10 15:22:31.012  0.039598  
            2019-07-10 15:22:31.016  0.042649

In [8]:
temp_df = curated_df.merge(log_events, how = "left", right_index = True, left_index = True)

In [9]:
t = temp_df.copy()

In [10]:
t['context'] = t['context'].ffill()
t['type'] = t['type'].ffill()
t['context'] = t['context'].fillna("Garbage")
t['type'] = t['type'].fillna("Garbage")

In [11]:
# t = t[pd.notnull(t['context'])]
t = t.reset_index().set_index(["Participant","type","context","Datetime"])
t.head()

RSP       PPG  \
Participant type    context Datetime                                      
0001        Garbage Garbage 2019-07-10 15:22:31.000 -1.263123  0.003052   
                            2019-07-10 15:22:31.004 -1.261902  0.003662   
                            2019-07-10 15:22:31.008 -1.264648  0.002136   
                            2019-07-10 15:22:31.012 -1.261902  0.003662   
                            2019-07-10 15:22:31.016 -1.262207  0.003357   

                                                        1-SKTA       ECG  \
Participant type    context Datetime                                       
0001        Garbage Garbage 2019-07-10 15:22:31.000  12.941162 -0.234833   
                            2019-07-10 15:22:31.004  12.945435 -0.226593   
                            2019-07-10 15:22:31.008  12.946045 -0.215607   
                            2019-07-10 15:22:31.012  12.945435 -0.206604   
                            2019-07-10 15:22:31.016  12.946045 -0.198364   

                                                          EDA  
Participant type    context Datetime                           
0001        Garbage Garbage 2019-07-10 15:22:31.000  0.041124  
                            2019-07-10 15:22:31.004  0.042649  
                            2019-07-10 15:22:31.008  0.044175  
                            2019-07-10 15:22:31.012  0.039598  
                            2019-07-10 15:22:31.016  0.042649

In [12]:
x_temp = t.copy()
x_temp.reset_index("Datetime", inplace = True)
x_new = pd.DataFrame(x_temp[["Datetime"]].groupby(["Participant","type","context"]).apply(lambda x: x['Datetime'].values[0]), columns = ["time_zero"])

In [13]:
x_new.head()
x_temp.head()

Datetime       RSP       PPG  \
Participant type    context                                               
0001        Garbage Garbage 2019-07-10 15:22:31.000 -1.263123  0.003052   
                    Garbage 2019-07-10 15:22:31.004 -1.261902  0.003662   
                    Garbage 2019-07-10 15:22:31.008 -1.264648  0.002136   
                    Garbage 2019-07-10 15:22:31.012 -1.261902  0.003662   
                    Garbage 2019-07-10 15:22:31.016 -1.262207  0.003357   

                                1-SKTA       ECG       EDA  
Participant type    context                                 
0001        Garbage Garbage  12.941162 -0.234833  0.041124  
                    Garbage  12.945435 -0.226593  0.042649  
                    Garbage  12.946045 -0.215607  0.044175  
                    Garbage  12.945435 -0.206604  0.039598  
                    Garbage  12.946045 -0.198364  0.042649

In [14]:
x_new2 = x_temp.merge(x_new, how = "left", right_index = True, left_index = True)
x_new2["Datetime_sub"] = x_new2["Datetime"] - x_new2["time_zero"]
x_new2.set_index(["Datetime_sub","Datetime"], append = True, inplace = True)
t = x_new2.copy()

In [15]:
t.head()

RSP  \
Participant type context        Datetime_sub    Datetime                            
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.000 -0.807800   
                                00:00:00.004000 2019-07-10 15:36:12.004 -0.794067   
                                00:00:00.008000 2019-07-10 15:36:12.008 -0.787659   
                                00:00:00.012000 2019-07-10 15:36:12.012 -0.779114   
                                00:00:00.016000 2019-07-10 15:36:12.016 -0.772705   

                                                                              PPG  \
Participant type context        Datetime_sub    Datetime                            
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.000  0.002441   
                                00:00:00.004000 2019-07-10 15:36:12.004  0.003052   
                                00:00:00.008000 2019-07-10 15:36:12.008  0.003357   
                                00:00:00.012000 2019-07-10 15:36:12.012  0.003052   
                                00:00:00.016000 2019-07-10 15:36:12.016  0.002747   

                                                                            1-SKTA  \
Participant type context        Datetime_sub    Datetime                             
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.000  12.976562   
                                00:00:00.004000 2019-07-10 15:36:12.004  12.979004   
                                00:00:00.008000 2019-07-10 15:36:12.008  12.977173   
                                00:00:00.012000 2019-07-10 15:36:12.012  12.979004   
                                00:00:00.016000 2019-07-10 15:36:12.016  12.978394   

                                                                              ECG  \
Participant type context        Datetime_sub    Datetime                            
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.000 -0.327148   
                                00:00:00.004000 2019-07-10 15:36:12.004 -0.327454   
                                00:00:00.008000 2019-07-10 15:36:12.008 -0.327148   
                                00:00:00.012000 2019-07-10 15:36:12.012 -0.327454   
                                00:00:00.016000 2019-07-10 15:36:12.016 -0.323334   

                                                                              EDA  \
Participant type context        Datetime_sub    Datetime                            
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.000  3.355333   
                                00:00:00.004000 2019-07-10 15:36:12.004  3.355333   
                                00:00:00.008000 2019-07-10 15:36:12.008  3.356858   
                                00:00:00.012000 2019-07-10 15:36:12.012  3.359910   
                                00:00:00.016000 2019-07-10 15:36:12.016  3.358384   

                                                                                  time_zero  
Participant type context        Datetime_sub    Datetime                                     
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.000 2019-07-10 15:36:12  
                                00:00:00.004000 2019-07-10 15:36:12.004 2019-07-10 15:36:12  
                                00:00:00.008000 2019-07-10 15:36:12.008 2019-07-10 15:36:12  
                                00:00:00.012000 2019-07-10 15:36:12.012 2019-07-10 15:36:12  
                                00:00:00.016000 2019-07-10 15:36:12.016 2019-07-10 15:36:12

# ECG

In [16]:
participants = t.index.get_level_values("Participant").unique()

In [17]:
good_peaks = joblib.load("/mnt/shared_drive/data/brain_therapy/MVP_Focused_Attention/FA_r_peaks_index.pkl")

Computing ECG Heart Rate and Heart Rate Variability

In [18]:
ecg_hr_features = pd.DataFrame([])
ecg_hrv_features = pd.DataFrame([])
time_diff_df = pd.DataFrame([])
for participant in participants:
    data = t.loc(axis = 0)[participant,:,:,:,:]
    ecg_peaks = good_peaks["BT-MVP-FA-{}".format(participant)]
    time_stamps = data.index.get_level_values('Datetime')[ecg_peaks]
    d = data.loc(axis = 0)[participant,:,:,:,time_stamps]
    
    
    #Computing time differences
    try:
        d['time_differences'] = np.concatenate([np.diff(time_stamps)/np.timedelta64(1,'ms'), [0]])
    except:
        d['time_differences'] = np.concatenate([np.diff(time_stamps)/np.timedelta64(1,'ms'), [0,0]])

    d = d[['time_differences']]
    #Taking out time differences that is more than a minute
    df = d[d.time_differences<60000]
    diff_mean = np.mean(df.time_differences)
    upper_threshold = diff_mean + (4*np.std(df.time_differences))
    lower_threshold = diff_mean - (4*np.std(df.time_differences))
    df = df[(df.time_differences<upper_threshold) & (df.time_differences>lower_threshold)]
    
    # Drop NaN contexts
    temp = df.reset_index("context")
    temp = temp[pd.notnull(temp['context'])]
#     display(temp)
    df = temp.reset_index().set_index(["Participant","type","context","Datetime","Datetime_sub"])
    time_diff_df = pd.concat([time_diff_df, df])
    

#     break

In [20]:
y_temp = time_diff_df.reset_index("Datetime_sub")
y_new = pd.DataFrame(y_temp[["Datetime_sub"]].groupby(["Participant","type","context"]).apply(lambda x: x['Datetime_sub'].values[0]),
                     columns = ["time_zero"])
y_new2 = y_temp.merge(y_new, how = "left", right_index = True, left_index = True)
y_new2["Datetime_sub"] = y_new2["Datetime_sub"] - y_new2["time_zero"]
y_new2 = y_new2.reset_index("Datetime").set_index(["Datetime_sub","Datetime"], append = True)
time_diff_new = y_new2.copy()
time_diff_new.head()

time_differences  \
Participant type context        Datetime_sub    Datetime                                    
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.700            1064.0   
                                00:00:01.064000 2019-07-10 15:36:13.764            1024.0   
                                00:00:02.088000 2019-07-10 15:36:14.788            1016.0   
                                00:00:03.104000 2019-07-10 15:36:15.804            1036.0   
                                00:00:04.140000 2019-07-10 15:36:16.840             940.0   

                                                                              time_zero  
Participant type context        Datetime_sub    Datetime                                 
0001        2D   StartEndBuffer 00:00:00        2019-07-10 15:36:12.700 00:00:00.700000  
                                00:00:01.064000 2019-07-10 15:36:13.764 00:00:00.700000  
                                00:00:02.088000 2019-07-10 15:36:14.788 00:00:00.700000  
                                00:00:03.104000 2019-07-10 15:36:15.804 00:00:00.700000  
                                00:00:04.140000 2019-07-10 15:36:16.840 00:00:00.700000

### ECG_HR features

Creating a separate dataframe for datetimes for ECG heart rate

In [26]:
temp_time_diff = time_diff_new.copy().reset_index("Datetime")
temp_time_diff.drop(["time_differences","time_zero"], axis = 1, inplace = True)

In [27]:
ecg_temp = temp_time_diff.groupby(["Participant","type","context", pd.Grouper(freq = "5s", level = "Datetime_sub")]).first()
ecg_temp.head()

Datetime
Participant type context        Datetime_sub                        
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700
                                00:00:05     2019-07-10 15:36:17.780
                                00:00:10     2019-07-10 15:36:23.604
                                00:00:15     2019-07-10 15:36:28.376
                                00:00:20     2019-07-10 15:36:33.016

In [28]:
ecg_hr_features = time_diff_new.groupby(["Participant","type","context",pd.Grouper(freq = "5s", level = "Datetime_sub")], 
                                        sort = True).mean()
ecg_hr_features['ecg_heart_rate'] = (60/ecg_hr_features['time_differences'])*1000
ecg_hr_features.drop("time_differences",axis = 1, inplace = True)
ecg_hr_features.head()

ecg_heart_rate
Participant type context        Datetime_sub                
0001        2D   StartEndBuffer 00:00:00           59.055118
                                00:00:05           61.813187
                                00:00:10           62.866723
                                00:00:15           64.655172
                                00:00:20           74.196208

Merging the features with its original Datetime

In [29]:
ecg_hr_features = ecg_hr_features.merge(ecg_temp, how = "left", right_index = True, left_index = True).\
                    set_index("Datetime", append = True)

In [30]:
ecg_hr_features.head()

ecg_heart_rate
Participant type context        Datetime_sub Datetime                               
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700       59.055118
                                00:00:05     2019-07-10 15:36:17.780       61.813187
                                00:00:10     2019-07-10 15:36:23.604       62.866723
                                00:00:15     2019-07-10 15:36:28.376       64.655172
                                00:00:20     2019-07-10 15:36:33.016       74.196208

### ECG_HRV features

Creating a separate dataframe for datetimes for ECG heart rate variability

In [31]:
ecg_hrv_temp = temp_time_diff.groupby(["Participant","type","context", pd.Grouper(freq = "20s", level = "Datetime_sub")]).first()
ecg_hrv_temp.head()

Datetime
Participant type context        Datetime_sub                        
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700
                                00:00:20     2019-07-10 15:36:33.016
                                00:00:40     2019-07-10 15:36:53.032
                                00:01:00     2019-07-10 15:37:13.032
                                00:01:20     2019-07-10 15:37:33.532

In [32]:
ecg_hrv_features = time_diff_new.groupby(["Participant","type","context",
                                          pd.Grouper(freq = "20s", level = "Datetime_sub")], sort = True).std()
ecg_hrv_features.rename(columns = {"time_differences":"ecg_hrv_std"}, inplace = True)
ecg_hrv_features.head()

ecg_hrv_std
Participant type context        Datetime_sub             
0001        2D   StartEndBuffer 00:00:00        70.691281
                                00:00:20        64.967976
                                00:00:40        55.276714
                                00:01:00        48.497066
                                00:01:20        91.838782

Merging the features with its original Datetime

In [33]:
ecg_hrv_features = ecg_hrv_features.merge(ecg_hrv_temp, how = "left", right_index = True, left_index = True).\
                    set_index("Datetime", append = True)

In [34]:
ecg_hrv_features.head()

ecg_hrv_std
Participant type context        Datetime_sub Datetime                            
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700    70.691281
                                00:00:20     2019-07-10 15:36:33.016    64.967976
                                00:00:40     2019-07-10 15:36:53.032    55.276714
                                00:01:00     2019-07-10 15:37:13.032    48.497066
                                00:01:20     2019-07-10 15:37:33.532    91.838782

# Skin Temperature

Creating a separate dataframe for datetimes for Skin Temperature

In [35]:
skt_temp = temp_time_diff.groupby(["Participant","type","context", pd.Grouper(freq = "10s", level = "Datetime_sub")]).first()
skt_temp.head()

Datetime
Participant type context        Datetime_sub                        
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700
                                00:00:10     2019-07-10 15:36:23.604
                                00:00:20     2019-07-10 15:36:33.016
                                00:00:30     2019-07-10 15:36:43.024
                                00:00:40     2019-07-10 15:36:53.032

In [36]:
skin_temperature = t[['1-SKTA']]
skin_temperature = skin_temperature.groupby(["Participant","type","context", pd.Grouper(freq = "10s", level = "Datetime_sub")]).mean()
skin_temperature.rename(columns = {"1-SKTA":"skin_temperature"}, inplace = True)
display(skin_temperature.head())

skin_temperature
Participant type context        Datetime_sub                  
0001        2D   StartEndBuffer 00:00:00             12.967830
                                00:00:10             12.959119
                                00:00:20             12.959495
                                00:00:30             12.967094
                                00:00:40             12.966464

Merging the features with its original Datetime

In [38]:
skin_temperature = skin_temperature.merge(skt_temp, how = "left", right_index = True, left_index = True).\
                    set_index("Datetime", append = True)

In [39]:
skin_temperature.head()

skin_temperature
Participant type context        Datetime_sub Datetime                                 
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700         12.967830
                                00:00:10     2019-07-10 15:36:23.604         12.959119
                                00:00:20     2019-07-10 15:36:33.016         12.959495
                                00:00:30     2019-07-10 15:36:43.024         12.967094
                                00:00:40     2019-07-10 15:36:53.032         12.966464

In [40]:
len(skin_temperature.loc(axis = 0)["0021","2D","StartLL",:])

13

# EDA

### EDA Mean

Creating a separate dataframe for datetimes of EDA

In [41]:
eda_temp = temp_time_diff.groupby(["Participant","type","context", pd.Grouper(freq = "30s", level = "Datetime_sub")]).first()
eda_temp.head()

Datetime
Participant type context          Datetime_sub                        
0001        2D   StartEndBuffer   00:00:00     2019-07-10 15:36:12.700
                                  00:00:30     2019-07-10 15:36:43.024
                                  00:01:00     2019-07-10 15:37:13.032
                                  00:01:30     2019-07-10 15:37:43.220
                 StartFrontBuffer 00:00:00     2019-07-10 15:25:32.828

In [42]:
#EDA MEAN
eda_features = t[['EDA']]
eda_features = eda_features.groupby(["Participant","type","context", pd.Grouper(freq = "30s", level = "Datetime_sub")]).mean()
eda_features.rename(columns = {"EDA":"eda_mean"}, inplace = True)
display(eda_features.head())

eda_mean
Participant type context          Datetime_sub          
0001        2D   StartEndBuffer   00:00:00      3.677715
                                  00:00:30      5.122383
                                  00:01:00      5.102888
                                  00:01:30      5.220675
                 StartFrontBuffer 00:00:00      5.467291

In [43]:
len(skin_temperature.loc(axis = 0)["0001","3D","StartLL",:])

13

### EDA Number of Peaks

In [44]:
def get_eda_features(eda):
    if len(eda > 15):
        try:
            features = nk.eda_process(eda = eda, sampling_rate = 250)
        except:
#             print("No peaks here")
            return 0
    else:
        return 0
    peaks = len(features['EDA']['SCR_Peaks_Indexes'])
    return peaks

In [45]:
#Number of peaks 
eda_peaks_df = pd.DataFrame([])
for participant in participants:
    data = t.loc(axis = 0)[participant,:,:,:]
#     display(data.head())
    d = data['EDA'].groupby(["Participant","type","context", pd.Grouper(freq = '30s', level = "Datetime_sub")]).apply(lambda x: get_eda_features(x))
    d = pd.DataFrame(d)
    eda_peaks_df = pd.concat([eda_peaks_df, d])
# t['EDA'].groupby(["Participant","type","context", pd.Grouper(freq = '30s', level = "Datetime_sub")]).apply(lambda x: get_eda_features(x))

In [46]:
eda_peaks_df.rename(columns = {"EDA":"eda_no_of_peaks"}, inplace = True)
eda_features = eda_features.merge(eda_peaks_df, how = "left",right_index = True, left_index = True)

In [47]:
eda_features.head()

eda_mean  eda_no_of_peaks
Participant type context          Datetime_sub                           
0001        2D   StartEndBuffer   00:00:00      3.677715                6
                                  00:00:30      5.122383                4
                                  00:01:00      5.102888                3
                                  00:01:30      5.220675                7
                 StartFrontBuffer 00:00:00      5.467291                3

### EDA Power Spectrum

In [48]:
def compute_power_spectra(signal, band):
    try:
        x = np.fft.fft(signal - np.nanmean(signal))
        x = np.abs(x)
        freq = np.fft.fftfreq(len(signal), d = 0.004)
    except:
        return 0
    low, high = np.array(band)
    vals = [i for i in x if high>i>low]
    idx = [list(x).index(i) for i in vals]

    frequencies = freq[idx]
    power = np.sum((frequencies/len(signal))**2)
    return power

In [49]:
def compute_frequency_band_power(signal, band, sampling_rate):
    freq, power = ss.periodogram(signal - np.nanmean(signal), sampling_rate)
    low_f1, low_f2, high_f1, high_f2 = np.array(band)
    lfp_idx  = np.where((freq>=low_f1) & (freq>=low_f2))[0]
    hfp_idx  = np.where((freq>=high_f1) & (freq>=high_f2))[0] 
    lfp = np.trapz(power[lfp_idx], x = freq[lfp_idx])
    hfp = np.trapz(power[hfp_idx], x = freq[hfp_idx])
    
    return [lfp, hfp, lfp/hfp]

In [50]:
#High Frequecy and Low Frequency
bands =  [0.045, 0.15, 0.15,0.25]
eda_powers_df = pd.DataFrame([])
for participant in participants:
    data = t.loc(axis = 0)[participant,:,:,:]
    eda_powers = data["EDA"].groupby(["Participant","type","context",pd.Grouper(freq = "30s", level = "Datetime_sub")]).\
                apply(lambda x:compute_frequency_band_power(x,bands,250) if len(x)>1 else pd.Series(np.nan))
    eda_powers = pd.DataFrame(eda_powers)
    eda_powers = pd.DataFrame(list(eda_powers["EDA"].values), index = eda_powers.index, 
                              columns = ["eda_lf","eda_hf","eda_lf_hf_ratio"])
    eda_powers_df = pd.concat([eda_powers_df, eda_powers])
   

In [51]:
eda_powers_df.head()

eda_lf    eda_hf  \
Participant type context          Datetime_sub                       
0001        2D   StartEndBuffer   00:00:00      0.037592  0.023888   
                                  00:00:30      0.001944  0.000539   
                                  00:01:00      0.007135  0.002519   
                                  00:01:30      0.006044  0.003774   
                 StartFrontBuffer 00:00:00      0.025428  0.019284   

                                                eda_lf_hf_ratio  
Participant type context          Datetime_sub                   
0001        2D   StartEndBuffer   00:00:00             1.573690  
                                  00:00:30             3.608991  
                                  00:01:00             2.832421  
                                  00:01:30             1.601694  
                 StartFrontBuffer 00:00:00             1.318618

In [52]:
eda_features = eda_features.merge(eda_powers_df, how = 'left', right_index = True, left_index = True)

In [53]:
eda_features.head()

eda_mean  eda_no_of_peaks  \
Participant type context          Datetime_sub                              
0001        2D   StartEndBuffer   00:00:00      3.677715                6   
                                  00:00:30      5.122383                4   
                                  00:01:00      5.102888                3   
                                  00:01:30      5.220675                7   
                 StartFrontBuffer 00:00:00      5.467291                3   

                                                  eda_lf    eda_hf  \
Participant type context          Datetime_sub                       
0001        2D   StartEndBuffer   00:00:00      0.037592  0.023888   
                                  00:00:30      0.001944  0.000539   
                                  00:01:00      0.007135  0.002519   
                                  00:01:30      0.006044  0.003774   
                 StartFrontBuffer 00:00:00      0.025428  0.019284   

                                                eda_lf_hf_ratio  
Participant type context          Datetime_sub                   
0001        2D   StartEndBuffer   00:00:00             1.573690  
                                  00:00:30             3.608991  
                                  00:01:00             2.832421  
                                  00:01:30             1.601694  
                 StartFrontBuffer 00:00:00             1.318618

Merging the features with its original Datetime

In [54]:
eda_features = eda_features.merge(eda_temp, how = "left", right_index = True, left_index = True).\
                    set_index("Datetime", append = True)

In [55]:
eda_features.head()

eda_mean  \
Participant type context          Datetime_sub Datetime                            
0001        2D   StartEndBuffer   00:00:00     2019-07-10 15:36:12.700  3.677715   
                                  00:00:30     2019-07-10 15:36:43.024  5.122383   
                                  00:01:00     2019-07-10 15:37:13.032  5.102888   
                                  00:01:30     2019-07-10 15:37:43.220  5.220675   
                 StartFrontBuffer 00:00:00     2019-07-10 15:25:32.828  5.467291   

                                                                        eda_no_of_peaks  \
Participant type context          Datetime_sub Datetime                                   
0001        2D   StartEndBuffer   00:00:00     2019-07-10 15:36:12.700                6   
                                  00:00:30     2019-07-10 15:36:43.024                4   
                                  00:01:00     2019-07-10 15:37:13.032                3   
                                  00:01:30     2019-07-10 15:37:43.220                7   
                 StartFrontBuffer 00:00:00     2019-07-10 15:25:32.828                3   

                                                                          eda_lf  \
Participant type context          Datetime_sub Datetime                            
0001        2D   StartEndBuffer   00:00:00     2019-07-10 15:36:12.700  0.037592   
                                  00:00:30     2019-07-10 15:36:43.024  0.001944   
                                  00:01:00     2019-07-10 15:37:13.032  0.007135   
                                  00:01:30     2019-07-10 15:37:43.220  0.006044   
                 StartFrontBuffer 00:00:00     2019-07-10 15:25:32.828  0.025428   

                                                                          eda_hf  \
Participant type context          Datetime_sub Datetime                            
0001        2D   StartEndBuffer   00:00:00     2019-07-10 15:36:12.700  0.023888   
                                  00:00:30     2019-07-10 15:36:43.024  0.000539   
                                  00:01:00     2019-07-10 15:37:13.032  0.002519   
                                  00:01:30     2019-07-10 15:37:43.220  0.003774   
                 StartFrontBuffer 00:00:00     2019-07-10 15:25:32.828  0.019284   

                                                                        eda_lf_hf_ratio  
Participant type context          Datetime_sub Datetime                                  
0001        2D   StartEndBuffer   00:00:00     2019-07-10 15:36:12.700         1.573690  
                                  00:00:30     2019-07-10 15:36:43.024         3.608991  
                                  00:01:00     2019-07-10 15:37:13.032         2.832421  
                                  00:01:30     2019-07-10 15:37:43.220         1.601694  
                 StartFrontBuffer 00:00:00     2019-07-10 15:25:32.828         1.318618

### Removing the "Start" and "Stop" from the events

In [56]:
def change_events(df, log_events, time_window):
    df = df.reset_index("context")
    s = [str(x).replace('Start','') for x in df['context'].values]
    s = [str(x).replace('Stop','') for x in s]
    s = [str(x).replace('nan','Garbage') for x in s]
    df['context'] = s
    df = df[df['context']!="FrontBuffer"]
    df = df[df['context']!="EndBuffer"]
    df = df.reset_index().set_index(["Participant","type","context","Datetime_sub"])
    return df

In [57]:
features_dataframe = {"ecg_hr":ecg_hr_features, "ecg_std":ecg_hrv_features, "skt":skin_temperature, "eda":eda_features}
time_windows = {"ecg_hr":"5s", "ecg_std":"20s", "skt":"10s", "eda": "30s"}
all_features = pd.DataFrame([])
flag = False
for index, data in features_dataframe.items():
    print(index)
    t = time_windows[index]
    f = change_events(data, log_events, t)
    display(f.head())
    


ecg_hr


Datetime  ecg_heart_rate
Participant type context Datetime_sub                                        
0001        2D   HH      00:00:00     2019-07-10 15:29:42.012       66.568047
                         00:00:05     2019-07-10 15:29:47.420       68.545316
                         00:00:10     2019-07-10 15:29:52.672       65.104167
                         00:00:15     2019-07-10 15:29:57.280       61.425061
                         00:00:20     2019-07-10 15:30:02.164       65.502183

ecg_std


Datetime  ecg_hrv_std
Participant type context Datetime_sub                                     
0001        2D   HH      00:00:00     2019-07-10 15:29:42.012    60.411289
                         00:00:20     2019-07-10 15:30:02.164    82.040176
                         00:00:40     2019-07-10 15:30:22.140    83.371593
                         00:01:00     2019-07-10 15:30:42.728   111.385606
                         00:01:20     2019-07-10 15:31:02.812    68.275632

skt


Datetime  \
Participant type context Datetime_sub                           
0001        2D   HH      00:00:00     2019-07-10 15:29:42.012   
                         00:00:10     2019-07-10 15:29:52.672   
                         00:00:20     2019-07-10 15:30:02.164   
                         00:00:30     2019-07-10 15:30:12.628   
                         00:00:40     2019-07-10 15:30:22.140   

                                       skin_temperature  
Participant type context Datetime_sub                    
0001        2D   HH      00:00:00             12.959742  
                         00:00:10             12.962748  
                         00:00:20             12.959219  
                         00:00:30             12.969762  
                         00:00:40             12.963512

eda


Datetime  eda_mean  \
Participant type context Datetime_sub                                     
0001        2D   HH      00:00:00     2019-07-10 15:29:42.012  3.248435   
                         00:00:30     2019-07-10 15:30:12.628  2.994607   
                         00:01:00     2019-07-10 15:30:42.728  2.745352   
                         00:01:30     2019-07-10 15:31:12.380  2.567717   
                         00:02:00     2019-07-10 15:31:43.088  2.327562   

                                       eda_no_of_peaks    eda_lf    eda_hf  \
Participant type context Datetime_sub                                        
0001        2D   HH      00:00:00                    6  0.000444  0.000203   
                         00:00:30                    9  0.001145  0.000660   
                         00:01:00                    4  0.001074  0.000334   
                         00:01:30                   11  0.001928  0.001121   
                         00:02:00                    2  0.000075  0.000062   

                                       eda_lf_hf_ratio  
Participant type context Datetime_sub                   
0001        2D   HH      00:00:00             2.186737  
                         00:00:30             1.736156  
                         00:01:00             3.215840  
                         00:01:30             1.719340  
                         00:02:00             1.212903

In [58]:
all_features = ecg_hr_features.merge(ecg_hrv_features, how = "left", right_index = True, left_index = True)
all_features.head()

ecg_heart_rate  \
Participant type context        Datetime_sub Datetime                                  
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700       59.055118   
                                00:00:05     2019-07-10 15:36:17.780       61.813187   
                                00:00:10     2019-07-10 15:36:23.604       62.866723   
                                00:00:15     2019-07-10 15:36:28.376       64.655172   
                                00:00:20     2019-07-10 15:36:33.016       74.196208   

                                                                      ecg_hrv_std  
Participant type context        Datetime_sub Datetime                              
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700    70.691281  
                                00:00:05     2019-07-10 15:36:17.780          NaN  
                                00:00:10     2019-07-10 15:36:23.604          NaN  
                                00:00:15     2019-07-10 15:36:28.376          NaN  
                                00:00:20     2019-07-10 15:36:33.016    64.967976

In [59]:
all_features = all_features.merge(skin_temperature, how = "left", right_index = True, left_index = True)
all_features.head()

ecg_heart_rate  \
Participant type context        Datetime_sub Datetime                                  
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700       59.055118   
                                00:00:05     2019-07-10 15:36:17.780       61.813187   
                                00:00:10     2019-07-10 15:36:23.604       62.866723   
                                00:00:15     2019-07-10 15:36:28.376       64.655172   
                                00:00:20     2019-07-10 15:36:33.016       74.196208   

                                                                      ecg_hrv_std  \
Participant type context        Datetime_sub Datetime                               
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700    70.691281   
                                00:00:05     2019-07-10 15:36:17.780          NaN   
                                00:00:10     2019-07-10 15:36:23.604          NaN   
                                00:00:15     2019-07-10 15:36:28.376          NaN   
                                00:00:20     2019-07-10 15:36:33.016    64.967976   

                                                                      skin_temperature  
Participant type context        Datetime_sub Datetime                                   
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700         12.967830  
                                00:00:05     2019-07-10 15:36:17.780               NaN  
                                00:00:10     2019-07-10 15:36:23.604         12.959119  
                                00:00:15     2019-07-10 15:36:28.376               NaN  
                                00:00:20     2019-07-10 15:36:33.016         12.959495

In [60]:
all_features = all_features.merge(eda_features, how = "left", right_index = True, left_index = True)
all_features.head(20)

ecg_heart_rate  \
Participant type context        Datetime_sub Datetime                                  
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700       59.055118   
                                00:00:05     2019-07-10 15:36:17.780       61.813187   
                                00:00:10     2019-07-10 15:36:23.604       62.866723   
                                00:00:15     2019-07-10 15:36:28.376       64.655172   
                                00:00:20     2019-07-10 15:36:33.016       74.196208   
                                00:00:25     2019-07-10 15:36:37.868       69.821567   
                                00:00:30     2019-07-10 15:36:43.024       66.617321   
                                00:00:35     2019-07-10 15:36:48.428       65.160730   
                                00:00:40     2019-07-10 15:36:53.032       66.371681   
                                00:00:45     2019-07-10 15:36:58.456       66.195940   
                                00:00:50     2019-07-10 15:37:02.988       62.240664   
                                00:00:55     2019-07-10 15:37:07.808       68.912711   
                                00:01:00     2019-07-10 15:37:13.032       63.721325   
                                00:01:05     2019-07-10 15:37:17.740       62.630480   
                                00:01:10     2019-07-10 15:37:23.488       62.656642   
                                00:01:15     2019-07-10 15:37:28.276       68.493151   
                                00:01:20     2019-07-10 15:37:33.532       71.315372   
                                00:01:25     2019-07-10 15:37:38.580       64.655172   
                                00:01:30     2019-07-10 15:37:43.220       61.983471   
                                00:01:35     2019-07-10 15:37:48.060       63.667233   

                                                                      ecg_hrv_std  \
Participant type context        Datetime_sub Datetime                               
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700    70.691281   
                                00:00:05     2019-07-10 15:36:17.780          NaN   
                                00:00:10     2019-07-10 15:36:23.604          NaN   
                                00:00:15     2019-07-10 15:36:28.376          NaN   
                                00:00:20     2019-07-10 15:36:33.016    64.967976   
                                00:00:25     2019-07-10 15:36:37.868          NaN   
                                00:00:30     2019-07-10 15:36:43.024          NaN   
                                00:00:35     2019-07-10 15:36:48.428          NaN   
                                00:00:40     2019-07-10 15:36:53.032    55.276714   
                                00:00:45     2019-07-10 15:36:58.456          NaN   
                                00:00:50     2019-07-10 15:37:02.988          NaN   
                                00:00:55     2019-07-10 15:37:07.808          NaN   
                                00:01:00     2019-07-10 15:37:13.032    48.497066   
                                00:01:05     2019-07-10 15:37:17.740          NaN   
                                00:01:10     2019-07-10 15:37:23.488          NaN   
                                00:01:15     2019-07-10 15:37:28.276          NaN   
                                00:01:20     2019-07-10 15:37:33.532    91.838782   
                                00:01:25     2019-07-10 15:37:38.580          NaN   
                                00:01:30     2019-07-10 15:37:43.220          NaN   
                                00:01:35     2019-07-10 15:37:48.060          NaN   

                                                                      skin_temperature  \
Participant type context        Datetime_sub Datetime                                    
0001        2D   StartEndBuffer 00:00:00     2019-07-10 15:36:12.700         12.96

In [61]:
# all_features.drop("context",axis = 1, inplace = True)
all_features.reset_index("context", inplace = True)
s = [str(x).replace('Start','') for x in all_features['context'].values]
s = [str(x).replace('Stop','') for x in s]
s = [str(x).replace('nan','Garbage') for x in s]
all_features['context'] = s
all_features = all_features[all_features["context"]!="EndBuffer"]
all_features = all_features[all_features['context']!="FrontBuffer"]
all_features = all_features.reset_index().set_index(["Participant","type","context","Datetime"])
all_features.head()

Datetime_sub  ecg_heart_rate  \
Participant type context Datetime                                               
0001        2D   HH      2019-07-10 15:29:42.012     00:00:00       66.568047   
                         2019-07-10 15:29:47.420     00:00:05       68.545316   
                         2019-07-10 15:29:52.672     00:00:10       65.104167   
                         2019-07-10 15:29:57.280     00:00:15       61.425061   
                         2019-07-10 15:30:02.164     00:00:20       65.502183   

                                                  ecg_hrv_std  \
Participant type context Datetime                               
0001        2D   HH      2019-07-10 15:29:42.012    60.411289   
                         2019-07-10 15:29:47.420          NaN   
                         2019-07-10 15:29:52.672          NaN   
                         2019-07-10 15:29:57.280          NaN   
                         2019-07-10 15:30:02.164    82.040176   

                                                  skin_temperature  eda_mean  \
Participant type context Datetime                                              
0001        2D   HH      2019-07-10 15:29:42.012         12.959742  3.248435   
                         2019-07-10 15:29:47.420               NaN       NaN   
                         2019-07-10 15:29:52.672         12.962748       NaN   
                         2019-07-10 15:29:57.280               NaN       NaN   
                         2019-07-10 15:30:02.164         12.959219       NaN   

                                                  eda_no_of_peaks    eda_lf  \
Participant type context Datetime                                             
0001        2D   HH      2019-07-10 15:29:42.012              6.0  0.000444   
                         2019-07-10 15:29:47.420              NaN       NaN   
                         2019-07-10 15:29:52.672              NaN       NaN   
                         2019-07-10 15:29:57.280              NaN       NaN   
                         2019-07-10 15:30:02.164              NaN       NaN   

                                                    eda_hf  eda_lf_hf_ratio  
Participant type context Datetime                                            
0001        2D   HH      2019-07-10 15:29:42.012  0.000203         2.186737  
                         2019-07-10 15:29:47.420       NaN              NaN  
                         2019-07-10 15:29:52.672       NaN              NaN  
                         2019-07-10 15:29:57.280       NaN              NaN  
                         2019-07-10 15:30:02.164       NaN              NaN

In [62]:
all_features = all_features.reset_index("Datetime").groupby(["Participant","type","context"], as_index = False).apply(lambda x: x.head(24))

In [63]:
all_features = all_features.reset_index().drop("level_0", axis = 1).\
        rename(columns = {"type":"Type"})
all_features = all_features[all_features['context']!="Garbage"]

all_features.set_index(["Participant","Type","context","Datetime"], inplace = True)
all_features.head()

Datetime_sub  ecg_heart_rate  \
Participant Type context Datetime                                               
0001        2D   HH      2019-07-10 15:29:42.012     00:00:00       66.568047   
                         2019-07-10 15:29:47.420     00:00:05       68.545316   
                         2019-07-10 15:29:52.672     00:00:10       65.104167   
                         2019-07-10 15:29:57.280     00:00:15       61.425061   
                         2019-07-10 15:30:02.164     00:00:20       65.502183   

                                                  ecg_hrv_std  \
Participant Type context Datetime                               
0001        2D   HH      2019-07-10 15:29:42.012    60.411289   
                         2019-07-10 15:29:47.420          NaN   
                         2019-07-10 15:29:52.672          NaN   
                         2019-07-10 15:29:57.280          NaN   
                         2019-07-10 15:30:02.164    82.040176   

                                                  skin_temperature  eda_mean  \
Participant Type context Datetime                                              
0001        2D   HH      2019-07-10 15:29:42.012         12.959742  3.248435   
                         2019-07-10 15:29:47.420               NaN       NaN   
                         2019-07-10 15:29:52.672         12.962748       NaN   
                         2019-07-10 15:29:57.280               NaN       NaN   
                         2019-07-10 15:30:02.164         12.959219       NaN   

                                                  eda_no_of_peaks    eda_lf  \
Participant Type context Datetime                                             
0001        2D   HH      2019-07-10 15:29:42.012              6.0  0.000444   
                         2019-07-10 15:29:47.420              NaN       NaN   
                         2019-07-10 15:29:52.672              NaN       NaN   
                         2019-07-10 15:29:57.280              NaN       NaN   
                         2019-07-10 15:30:02.164              NaN       NaN   

                                                    eda_hf  eda_lf_hf_ratio  
Participant Type context Datetime                                            
0001        2D   HH      2019-07-10 15:29:42.012  0.000203         2.186737  
                         2019-07-10 15:29:47.420       NaN              NaN  
                         2019-07-10 15:29:52.672       NaN              NaN  
                         2019-07-10 15:29:57.280       NaN              NaN  
                         2019-07-10 15:30:02.164       NaN              NaN

In [64]:
d = all_features.copy()
d.head()

Datetime_sub  ecg_heart_rate  \
Participant Type context Datetime                                               
0001        2D   HH      2019-07-10 15:29:42.012     00:00:00       66.568047   
                         2019-07-10 15:29:47.420     00:00:05       68.545316   
                         2019-07-10 15:29:52.672     00:00:10       65.104167   
                         2019-07-10 15:29:57.280     00:00:15       61.425061   
                         2019-07-10 15:30:02.164     00:00:20       65.502183   

                                                  ecg_hrv_std  \
Participant Type context Datetime                               
0001        2D   HH      2019-07-10 15:29:42.012    60.411289   
                         2019-07-10 15:29:47.420          NaN   
                         2019-07-10 15:29:52.672          NaN   
                         2019-07-10 15:29:57.280          NaN   
                         2019-07-10 15:30:02.164    82.040176   

                                                  skin_temperature  eda_mean  \
Participant Type context Datetime                                              
0001        2D   HH      2019-07-10 15:29:42.012         12.959742  3.248435   
                         2019-07-10 15:29:47.420               NaN       NaN   
                         2019-07-10 15:29:52.672         12.962748       NaN   
                         2019-07-10 15:29:57.280               NaN       NaN   
                         2019-07-10 15:30:02.164         12.959219       NaN   

                                                  eda_no_of_peaks    eda_lf  \
Participant Type context Datetime                                             
0001        2D   HH      2019-07-10 15:29:42.012              6.0  0.000444   
                         2019-07-10 15:29:47.420              NaN       NaN   
                         2019-07-10 15:29:52.672              NaN       NaN   
                         2019-07-10 15:29:57.280              NaN       NaN   
                         2019-07-10 15:30:02.164              NaN       NaN   

                                                    eda_hf  eda_lf_hf_ratio  
Participant Type context Datetime                                            
0001        2D   HH      2019-07-10 15:29:42.012  0.000203         2.186737  
                         2019-07-10 15:29:47.420       NaN              NaN  
                         2019-07-10 15:29:52.672       NaN              NaN  
                         2019-07-10 15:29:57.280       NaN              NaN  
                         2019-07-10 15:30:02.164       NaN              NaN

In [65]:
all_features = all_features.reset_index().set_index(["Participant","Datetime","Type","context"]).sort_index(sort_remaining = True)
all_features = all_features.reset_index().set_index(["Participant","Type","context","Datetime"])

In [66]:
all_features.head(3)

Datetime_sub  ecg_heart_rate  \
Participant Type context Datetime                                               
0001        2D   LL      2019-07-10 15:27:32.344     00:00:00       66.617321   
                         2019-07-10 15:27:37.748     00:00:05       68.807339   
                         2019-07-10 15:27:42.980     00:00:10       61.525841   

                                                  ecg_hrv_std  \
Participant Type context Datetime                               
0001        2D   LL      2019-07-10 15:27:32.344    75.818192   
                         2019-07-10 15:27:37.748          NaN   
                         2019-07-10 15:27:42.980          NaN   

                                                  skin_temperature  eda_mean  \
Participant Type context Datetime                                              
0001        2D   LL      2019-07-10 15:27:32.344         12.958056   4.31652   
                         2019-07-10 15:27:37.748               NaN       NaN   
                         2019-07-10 15:27:42.980         12.968002       NaN   

                                                  eda_no_of_peaks    eda_lf  \
Participant Type context Datetime                                             
0001        2D   LL      2019-07-10 15:27:32.344              4.0  0.003565   
                         2019-07-10 15:27:37.748              NaN       NaN   
                         2019-07-10 15:27:42.980              NaN       NaN   

                                                    eda_hf  eda_lf_hf_ratio  
Participant Type context Datetime                                            
0001        2D   LL      2019-07-10 15:27:32.344  0.002215         1.609201  
                         2019-07-10 15:27:37.748       NaN              NaN  
                         2019-07-10 15:27:42.980       NaN              NaN

In [67]:
all_features.index.get_level_values("context").unique()

Index(['LL', 'HH', 'LH', 'HL'], dtype='object', name='context')

In [75]:
t3 = pd.DataFrame([])
for participant in participants:
    t2 = pd.DataFrame([])
    counter = 1
    sessions = all_features.loc(axis = 0)[participant,:,:,:].index.get_level_values('Type').unique()
    for k, session in enumerate(sessions):
        t1 = pd.DataFrame([])
        events = all_features.loc(axis = 0)[participant,session,:,:].index.get_level_values('context').unique()
        for j,event in enumerate(events):
            d = all_features.loc(axis = 0)[participant,session,event,:]
            d['feature_time'] = np.arange(0,len(d['ecg_heart_rate'].values))
            d['event_order'] = j+1
            d['event_session_order'] = counter
            counter+=1
            t1 = pd.concat([t1,d])
#             break
#             display(t1)
        t1['session_order'] = k+1
#         print(k)
        t2 = pd.concat([t2,t1])
#         break
    t3 = pd.concat([t3,t2])
#     break

In [76]:
t3.drop("Datetime_sub", axis = 1, inplace = True)

In [77]:
t3.tail(30)

ecg_heart_rate  ecg_hrv_std  \
Participant Type context Datetime                                               
0021        3D   HH      2019-08-06 13:42:31.308       67.064083          NaN   
                         2019-08-06 13:42:36.676       65.217391          NaN   
                         2019-08-06 13:42:41.276       62.456627    42.122477   
                         2019-08-06 13:42:47.040       67.385445          NaN   
                         2019-08-06 13:42:51.492       65.934066          NaN   
                         2019-08-06 13:42:56.952       67.934783          NaN   
                 LL      2019-08-06 13:43:11.748       71.428571    35.730099   
                         2019-08-06 13:43:16.788       73.068894          NaN   
                         2019-08-06 13:43:22.536       73.409462          NaN   
                         2019-08-06 13:43:27.440       73.289902          NaN   
                         2019-08-06 13:43:32.352       70.699136    31.792352   
                         2019-08-06 13:43:37.444       74.937552          NaN   
                         2019-08-06 13:43:42.248       73.111292          NaN   
                         2019-08-06 13:43:47.172       77.054795          NaN   
                         2019-08-06 13:43:51.844       75.000000    35.413340   
                         2019-08-06 13:43:57.444       76.077768          NaN   
                         2019-08-06 13:44:02.176       82.031250          NaN   
                         2019-08-06 13:44:07.296       80.285459          NaN   
                         2019-08-06 13:44:11.780       87.272727    38.521622   
                         2019-08-06 13:44:17.280       78.740157          NaN   
                         2019-08-06 13:44:21.852       80.213904          NaN   
                         2019-08-06 13:44:27.088       85.227273          NaN   
                         2019-08-06 13:44:32.016       83.003953    27.203303   
                         2019-08-06 13:44:37.076       86.206897          NaN   
                         2019-08-06 13:44:41.948       90.566038          NaN   
                         2019-08-06 13:44:47.248       89.897260          NaN   
                         2019-08-06 13:44:51.920       86.348684    28.053432   
                         2019-08-06 13:44:56.784       81.143740          NaN   
                         2019-08-06 13:45:01.960       85.227273          NaN   
                         2019-08-06 13:45:06.888       89.020772          NaN   

                                                  skin_temperature   eda_mean  \
Participant Type context Datetime                                               
0021        3D   HH      2019-08-06 13:42:31.308         22.520994  10.390384   
                         2019-08-06 13:42:36.676               NaN        NaN   
                         2019-08-06 13:42:41.276         22.521035        NaN   
                         2019-08-06 13:42:47.040               NaN        NaN   
                         2019-08-06 13:42:51.492         22.521035        NaN   
                         2019-08-06 13:42:56.952               NaN        NaN   
                 LL      2019-08-06 13:43:11.748         22.543603  10.355387   
                         2019-08-06 13:43:16.788               NaN        NaN   
                         2019-08-06 13:43:22.536         22.550411        NaN   
                         2019-08-06 13:43:27.440               NaN        NaN   
                         2019-08-06 13:43:32.352         22.550353        NaN   
                         2019-08-06 13:43:37.444               NaN        NaN   
                         2019-08-06 13:43:42.248         22.574708  10.431909   
                         2019-08-06 13:43:47.172               NaN        NaN   
                         2019-08-06 13:43:51.844         22.579786        NaN   
                         2019-08-06 13:43:57.444               NaN        NaN   

In [81]:
feature_names = all_features.columns[1:len(all_features.columns)]
feature_names

Index(['ecg_heart_rate', 'ecg_hrv_std', 'skin_temperature', 'eda_mean',
       'eda_no_of_peaks', 'eda_lf', 'eda_hf', 'eda_lf_hf_ratio'],
      dtype='object')

In [118]:
t3.head()

ecg_heart_rate  ecg_hrv_std  \
Participant Type context Datetime                                               
0001        2D   LL      2019-07-10 15:27:32.344       66.617321    75.818192   
                         2019-07-10 15:27:37.748       68.807339          NaN   
                         2019-07-10 15:27:42.980       61.525841          NaN   
                         2019-07-10 15:27:47.856       65.047702          NaN   
                         2019-07-10 15:27:52.468       61.174551    86.211057   

                                                  skin_temperature  eda_mean  \
Participant Type context Datetime                                              
0001        2D   LL      2019-07-10 15:27:32.344         12.958056   4.31652   
                         2019-07-10 15:27:37.748               NaN       NaN   
                         2019-07-10 15:27:42.980         12.968002       NaN   
                         2019-07-10 15:27:47.856               NaN       NaN   
                         2019-07-10 15:27:52.468         12.967983       NaN   

                                                  eda_no_of_peaks    eda_lf  \
Participant Type context Datetime                                             
0001        2D   LL      2019-07-10 15:27:32.344              4.0  0.003565   
                         2019-07-10 15:27:37.748              NaN       NaN   
                         2019-07-10 15:27:42.980              NaN       NaN   
                         2019-07-10 15:27:47.856              NaN       NaN   
                         2019-07-10 15:27:52.468              NaN       NaN   

                                                    eda_hf  eda_lf_hf_ratio  \
Participant Type context Datetime                                             
0001        2D   LL      2019-07-10 15:27:32.344  0.002215         1.609201   
                         2019-07-10 15:27:37.748       NaN              NaN   
                         2019-07-10 15:27:42.980       NaN              NaN   
                         2019-07-10 15:27:47.856       NaN              NaN   
                         2019-07-10 15:27:52.468       NaN              NaN   

                                                  feature_time  event_order  \
Participant Type context Datetime                                             
0001        2D   LL      2019-07-10 15:27:32.344             0            1   
                         2019-07-10 15:27:37.748             1            1   
                         2019-07-10 15:27:42.980             2            1   
                         2019-07-10 15:27:47.856             3            1   
                         2019-07-10 15:27:52.468             4            1   

                                                  event_session_order  \
Participant Type context Datetime                                       
0001        2D   LL      2019-07-10 15:27:32.344                    1   
                         2019-07-10 15:27:37.748                    1   
                         2019-07-10 15:27:42.980                    1   
                         2019-07-10 15:27:47.856                    1   
                         2019-07-10 15:27:52.468                    1   

                                                  session_order  
Participant Type context Datetime                                
0001        2D   LL      2019-07-10 15:27:32.344              1  
                         2019-07-10 15:27:37.748              1  
                         2019-07-10 15:27:42.980              1  
                         2019-07-10 15:27:47.856              1  
                         2019-07-10 15:27:52.468              1

In [ ]:
data_full = t3.copy()
cleaned_features = pd.DataFrame([])
for participant in participants:
    data = data_full.loc(axis =  0)[participant,:,:,:]
    for feature in feature_names:
        values = data[feature].values
        values_mean = np.nanmean(values)
        upper_threshold = values_mean + (3*np.std(values))
        lower_threshold = values_mean - (3*np.std(values))
        v = [x if (x > lower_threshold and x < upper_threshold) else np.nan for x in values]
        data[feature] = v
    cleaned_features = pd.concat([cleaned_features,data])
    

In [ ]:
data.head(30)

In [133]:
data_full = data_full.groupby("Participant").apply(lambda x: filter_features(x))

In [135]:
def filter_features(data):
    features_names = ['ecg_heart_rate', 'ecg_hrv_std', 'skin_temperature', 'eda_mean',
                       'eda_no_of_peaks', 'eda_lf', 'eda_hf', 'eda_lf_hf_ratio']
    for feature in feature_names:
        values = data[[feature]].values
        values_mean = np.nanmean(values)
        upper_threshold = values_mean + (3*np.nanstd(values))
        lower_threshold = values_mean - (3*np.nanstd(values))
        v = [x[0] if (x > lower_threshold and x < upper_threshold) else np.nan for x in values]
        data[feature] = v
    return data

In [136]:
joblib.dump(data_full, "/mnt/shared_drive/data/brain_therapy/new_features.pkl")

['/mnt/shared_drive/data/brain_therapy/new_features.pkl']

In [ ]:
feature_anove = pd.read_csv("/mnt/bkt_prd_dsv_bbrain_therapy_raw")

In [458]:
t3.loc(axis = 0)["0010","2D"].index.get_level_values("context").unique()

Index(['HH', 'LL', 'LH', 'HL'], dtype='object', name='context')

In [137]:
t3.head(30)

ecg_heart_rate  ecg_hrv_std  \
Participant Type context Datetime                                               
0001        2D   LL      2019-07-10 15:27:32.344       66.617321    75.818192   
                         2019-07-10 15:27:37.748       68.807339          NaN   
                         2019-07-10 15:27:42.980       61.525841          NaN   
                         2019-07-10 15:27:47.856       65.047702          NaN   
                         2019-07-10 15:27:52.468       61.174551    86.211057   
                         2019-07-10 15:27:57.372       66.469719          NaN   
                         2019-07-10 15:28:02.788       61.475410          NaN   
                         2019-07-10 15:28:07.668       66.815145          NaN   
                         2019-07-10 15:28:13.056       66.785396   140.895386   
                         2019-07-10 15:28:17.548       68.233510          NaN   
                         2019-07-10 15:28:22.824       85.158151          NaN   
                         2019-07-10 15:28:27.756       58.962264          NaN   
                         2019-07-10 15:28:32.844       63.131313    95.092391   
                         2019-07-10 15:28:37.596       65.982405          NaN   
                         2019-07-10 15:28:43.052       72.289157          NaN   
                         2019-07-10 15:28:48.032       58.094500          NaN   
                         2019-07-10 15:28:53.196       62.086093    72.864459   
                         2019-07-10 15:28:58.028       65.160730          NaN   
                         2019-07-10 15:29:02.632       70.921986          NaN   
                         2019-07-10 15:29:07.708       68.912711          NaN   
                         2019-07-10 15:29:12.932       72.815534    92.394037   
                         2019-07-10 15:29:17.876       69.984448          NaN   
                         2019-07-10 15:29:23.020       63.291139          NaN   
                         2019-07-10 15:29:27.760       65.274151          NaN   
                 HH      2019-07-10 15:29:42.012       66.568047    60.411289   
                         2019-07-10 15:29:47.420       68.545316          NaN   
                         2019-07-10 15:29:52.672       65.104167          NaN   
                         2019-07-10 15:29:57.280       61.425061          NaN   
                         2019-07-10 15:30:02.164       65.502183    82.040176   
                         2019-07-10 15:30:07.660       60.386473          NaN   

                                                  skin_temperature  eda_mean  \
Participant Type context Datetime                                              
0001        2D   LL      2019-07-10 15:27:32.344         12.958056  4.316520   
                         2019-07-10 15:27:37.748               NaN       NaN   
                         2019-07-10 15:27:42.980         12.968002       NaN   
                         2019-07-10 15:27:47.856               NaN       NaN   
                         2019-07-10 15:27:52.468         12.967983       NaN   
                         2019-07-10 15:27:57.372               NaN       NaN   
                         2019-07-10 15:28:02.788         12.967984  4.445775   
                         2019-07-10 15:28:07.668               NaN       NaN   
                         2019-07-10 15:28:13.056         12.968006       NaN   
                         2019-07-10 15:28:17.548               NaN       NaN   
                         2019-07-10 15:28:22.824         12.967968       NaN   
                         2019-07-10 15:28:27.756               NaN       NaN   
                         2019-07-10 15:28:32.844         12.967958  3.959713   
                         2019-07-10 15:28:37.596               NaN       NaN   
                         2019-07-10 15:28:43.052         12.967907       NaN   
                         2019-07-10 15:28:48.032               NaN       NaN   
                 